In [38]:
%run utils

In [39]:
from torch import nn

class ConvBn(nn.Module):
    def __init__(self, *args, **kwargs):
        super(ConvBn, self).__init__()
        self.conv = nn.Conv2d(*args, **kwargs)
        self.bn = nn.BatchNorm2d(self.conv.out_channels)
        nn.init.kaiming_normal_(self.conv.weight)
        nn.init.zeros_(self.conv.bias)

    def forward(self, x):
        return self.bn(self.conv(x))

class ConvBnRelu(nn.Module):
    def __init__(self, *args, **kwargs):
        super(ConvBnRelu, self).__init__()
        self.conv = ConvBn(*args, **kwargs)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.relu(self.conv(x))

In [40]:
class ResUnit(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResUnit, self).__init__()
        self.conv1 = ConvBnRelu(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.conv2 = ConvBn(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.downsample = None
        if in_channels != out_channels or stride != 1:
            self.downsample = nn.Sequential(
                ConvBn(in_channels, out_channels, kernel_size=1, stride=stride),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        identity = x
        out = self.conv1(x)
        out = self.conv2(out)
        if self.downsample is not None:
            identity = self.downsample(x)
        out += identity
        return nn.ReLU()(out)


unit = ResUnit(64, 128, stride=2)
print(unit)

ResUnit(
  (conv1): ConvBnRelu(
    (conv): ConvBn(
      (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (relu): ReLU()
  )
  (conv2): ConvBn(
    (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (downsample): Sequential(
    (0): ConvBn(
      (conv): Conv2d(64, 128, kernel_size=(1, 1), stride=(2, 2))
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)


In [41]:
class ResBlock(nn.Module):
    def __init__(self, in_channels, out_channels, num_units, stride=1):
        super(ResBlock, self).__init__()
        layers = []
        layers.append(ResUnit(in_channels, out_channels, stride=stride))
        for i in range(num_units - 1):
            layers.append(ResUnit(out_channels, out_channels, stride=1))
        self.blocks = nn.Sequential(*layers)

    def forward(self, x):
        return self.blocks(x)

block = ResBlock(64, 128, 2)
print(block)

ResBlock(
  (blocks): Sequential(
    (0): ResUnit(
      (conv1): ConvBnRelu(
        (conv): ConvBn(
          (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (relu): ReLU()
      )
      (conv2): ConvBn(
        (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (downsample): Sequential(
        (0): ConvBn(
          (conv): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1))
          (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): ResUnit(
      (conv1): ConvBnRelu(
        (conv): ConvBn(
          (conv): Conv2d(128, 128, kernel_size=(3, 3), str

In [ ]:
class ResNet(nn.Module):
    def __init__(self):
        super(ResNet, self).__init__()
        self.conv1 = ConvBnRelu(3, 64, kernel_size=3, stride=1, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.block1 = ResBlock(64, 64, 2)
        self.block2 = ResBlock(64, 128, 2, stride=2)
        self.block3 = ResBlock(128, 256, 2, stride=2)
        self.block4 = ResBlock(256, 512, 2, stride=2)
        self.pool2 = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.pool2(x)
        x = nn.Flatten()(x)
        x = self.fc(x)
        return x


In [ ]:
from torchinfo import summary

net = ResNet()
summary(net, (1, 3, 32, 32))

In [ ]:
from torchvision import datasets
from utils import train_val_split
from utils import PackDataset
from torchvision import transforms

full = datasets.CIFAR10(root="./data", train=True, download=True)
test = datasets.CIFAR10(root="./data", train=False, download=True)
train, valid = train_val_split(full, seed=666)

train_data = PackDataset(train, transform=transforms.ToTensor())
valid_data = PackDataset(valid, transform=transforms.ToTensor())
test_data = PackDataset(test, transform=transforms.ToTensor())

In [ ]:
import json
from utils import control_callbacks
from sklearn.model_selection import ParameterGrid
import torch
from skorch import NeuralNetClassifier
from skorch.helper import predefined_split

In [ ]:
epochs = 20
param_grid = {
    'lr': [0.01, 0.005, 0.001, 0.0005, 0.0001],
}

results = {
    'best_params': None,
    'best_acc': 0.0,
    'all_results': []
}

calls = control_callbacks(epochs, check_dir='./data/alex-checkpoints', show_bar=False)
for params in ParameterGrid(param_grid):
    print(f"\nTraining with params: {params}")
    net = NeuralNetClassifier(
        ResNet,
        criterion=nn.CrossEntropyLoss,
        optimizer=torch.optim.Adam,
        lr=params['lr'],
        batch_size=2048,
        max_epochs=epochs,
        train_split=predefined_split(valid_data),
        device='cuda' if torch.cuda.is_available() else 'cpu',
        callbacks=calls,
        classes=list(range(10)),
    )
    net.fit(X=train_data, y=None)
    valid_acc = max(net.history[:, 'valid_acc'])
    current_result = {'params': params, 'valid_acc': valid_acc}
    results['all_results'].append(current_result)

    if valid_acc > results['best_acc']:
        results['best_acc'] = valid_acc
        results['best_params'] = params

    print(f"\nBest params: {results['best_params']}, best acc: {results['best_acc']}")

with open('./data/hyperparam_results.json', 'w') as f:
    json.dump(results, f, indent=2)